In [6]:
import pandas as pd
test_df = pd.read_csv("/Users/datou/PycharmProjects/model/HengyangSta/HY_ensemble/data/衡阳站多模型验证期结果.csv",
                      parse_dates=["Date"])
test_df_copy = test_df
test_df_copy = test_df_copy.iloc[:, :2]
test_df.reset_index(drop=True)
test_df = test_df.set_index('Date')
"""
读取训练数据
"""
train_df = pd.read_csv("/Users/datou/PycharmProjects/model/HengyangSta/HY_ensemble/data/衡阳站多模型率定期结果.csv",
                       parse_dates=["Date"])
train_df_copy = train_df
train_df_copy = train_df_copy.iloc[:, :2]
train_df.reset_index(drop=True)
train_df = train_df.set_index('Date')
test_df

,Qobs,XAJ,TVGM,TANK,GR4J,BP,SVR,RNN,LSTM
Date,,,,,,,,,
1996-11-25,391,225.033998,0.000000,0.0000,735.3500,412.51212,452.107690,330.67706,359.58588
1996-11-26,383,254.349212,1.243738,222.5366,677.0830,427.12274,447.654346,335.34006,359.64227
1996-11-27,292,238.780517,9.174156,358.5957,500.3750,427.80844,427.297602,334.46700,353.75110
1996-11-28,316,215.054604,16.085507,232.6648,393.1510,357.89010,300.937053,258.40210,265.39352
1996-11-29,309,192.677854,37.273250,190.3943,322.3010,402.05334,401.121845,293.06906,333.27728
...,...,...,...,...,...,...,...,...,...
2001-12-27,766,303.171266,201.602220,632.9010,325.1101,705.56226,796.338839,617.83813,653.80597
2001-12-28,653,280.365424,164.880794,597.6152,317.9625,683.90850,816.775477,648.15950,669.88380
2001-12-29,536,260.064636,138.430022,561.9181,311.1167,565.22125,622.037698,570.21160,546.56840


In [7]:
# 导入进行线性回归的包

from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
import numpy as np
from tool import Indicator
"""
案例一：XAJ+GR4J
"""
# 处理数据
trainX_xaj = train_df.iloc[:, 1]
trainX_gr4j = train_df.iloc[:, 4]
trainy1 = train_df.iloc[:, :1]
trainx1 = pd.merge(trainX_xaj, trainX_gr4j, on="Date", how="left")
train_data1=pd.merge(trainy1,trainx1,on="Date",how="left")
testX_xaj = test_df.iloc[:, 1]
testX_gr4j = test_df.iloc[:,4]
testy1 = test_df.iloc[:, :1]
testx1 = pd.merge(testX_xaj, testX_gr4j, on="Date", how="left")
test_data1=pd.merge(testy1,testx1,on="Date",how="left")

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled1 = scaler.fit_transform(train_data1)
df_for_testing_scaled1 = scaler.transform(test_data1)

# 划分数据
trainX1 = df_for_training_scaled1[:, 1:]
trainY1 = df_for_training_scaled1[:, :1]
testX1 = df_for_testing_scaled1[:, 1:]
testY1 = df_for_testing_scaled1[:, :1]


# XGBoost
parameters = {
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.15, 0.025],
    'n_estimators': [50, 100, 200],
    'subsample':[ 0.7, 0.8, 0.9, 1]
}
xlf = xgb.XGBRegressor()
grid_search = GridSearchCV(xlf, param_grid=parameters, cv=5)
grid_search = grid_search.fit(trainX1, trainY1)
# 拟合数据集
model1 = grid_search.best_estimator_
print("最佳参数组合："+str(grid_search.best_params_))

train_predict1 = model1.predict(trainX1)
train_predict1 = pd.DataFrame(train_predict1)
train_predict1.columns = ["XGBoost1"]

test_predict1 = model1.predict(testX1)
test_predict1 = pd.DataFrame(test_predict1)
test_predict1.columns = ["XGBoost1"]


# 逆缩放过程
# 改变形状来进行逆缩放，逆变换后的第一列是我们需要的，所以我们在最后使用了 → [:,0]。
y_pred_copy = np.repeat(test_predict1,len(test_data1.columns), axis=-1)
test_pred1 = scaler.inverse_transform(np.reshape(y_pred_copy, (len(test_predict1),len(test_data1.columns))))[:, 0]
test_pred_copy1 = test_pred1
test_pred1 = pd.DataFrame(test_pred1)
test_pred1.columns = ["XGBoost1"]

prediction_copies_array = np.repeat(train_predict1,len(test_data1.columns), axis=-1)
train_pred1 = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(train_predict1),len(test_data1.columns))))[:, 0]
train_pred_copy1 = train_pred1
train_pred1=pd.DataFrame(train_pred1)
train_pred1.columns=['XGBoost1']


# 计算nash效率系数
print("*******************率定期*******************")
Indicator.eval(train_pred1['XGBoost1'], train_df_copy['Qobs'])
print("*******************验证期*******************")
Indicator.eval(test_pred1['XGBoost1'], test_df_copy['Qobs'])

最佳参数组合：{'learning_rate': 0.15, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1}
*******************率定期*******************
MAE:265.76307672937924
RMSE:436.99731437687245
NSE:0.8949155768062546
RE:0.16581722602923338
Qmaxe:0.054651292945906434
Qmine:1.0742216564360119
*******************验证期*******************
MAE:383.05627737900164
RMSE:640.245348932015
NSE:0.8116167905857318
RE:7.460804106763665
Qmaxe:0.20938817223837208
Qmine:1.5373973661256068


(383.05627737900164,
 640.245348932015,
 0.8116167905857318,
 7.460804106763665,
 0.20938817223837208,
 1.5373973661256068)

In [8]:
"""
案例二：LSTM+RNN
"""
# 处理数据
trainX_LSTM = train_df.iloc[:, 7]
trainX_RNN = train_df.iloc[:, 8]
trainy2 = train_df.iloc[:, :1]
trainx2 = pd.merge(trainX_LSTM, trainX_RNN, on="Date", how="left")
testX_LSTM = test_df.iloc[:, 7]
testX_RNN = test_df.iloc[:, 8]
testy2 = test_df.iloc[:, :1]
testx2 = pd.merge(testX_LSTM, testX_RNN, on="Date", how="left")
train_data2=pd.merge(trainy2,trainx2,on="Date",how="left")
test_data2=pd.merge(testy2,testx2,on="Date",how="left")

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled2 = scaler.fit_transform(train_data2)
df_for_testing_scaled2 = scaler.transform(test_data2)

# 划分数据
trainX2 = df_for_training_scaled2[:, 1:]
trainY2 = df_for_training_scaled2[:, :1]
testX2 = df_for_testing_scaled2[:, 1:]
testY2 = df_for_testing_scaled2[:, :1]


# 随机森林回归器
parameters = {
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.15, 0.025],
    'n_estimators': [50, 100, 200],
    'subsample':[ 0.7, 0.8, 0.9, 1]
}
xlf = xgb.XGBRegressor()
grid_search = GridSearchCV(xlf, param_grid=parameters, cv=5)
grid_search = grid_search.fit(trainX2, trainY2)
# 拟合数据集
model1 = grid_search.best_estimator_
print("最佳参数组合："+str(grid_search.best_params_))

train_predict2 = model1.predict(trainX2)
train_predict2 = pd.DataFrame(train_predict2)
train_predict2.columns = ["XGBoost2"]

test_predict2 = model1.predict(testX2)
test_predict2 = pd.DataFrame(test_predict2)
test_predict2.columns = ["XGBoost2"]


# 逆缩放过程
# 改变形状来进行逆缩放，逆变换后的第一列是我们需要的，所以我们在最后使用了 → [:,0]。
y_pred_copy = np.repeat(test_predict2,len(test_data2.columns), axis=-1)
test_pred2 = scaler.inverse_transform(np.reshape(y_pred_copy, (len(test_predict2),len(test_data2.columns))))[:, 0]
test_pred_copy2 = test_pred2
test_pred2 = pd.DataFrame(test_pred2)
test_pred2.columns = ["XGBoost2"]

prediction_copies_array = np.repeat(train_predict2,len(test_data2.columns), axis=-1)
train_pred2 = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(train_predict2),len(test_data2.columns))))[:, 0]
train_pred_copy2 = train_pred2
train_pred2=pd.DataFrame(train_pred2)
train_pred2.columns=['XGBoost2']


# 计算nash效率系数
print("*******************率定期*******************")
Indicator.eval(train_pred2['XGBoost2'], train_df_copy['Qobs'])
print("*******************验证期*******************")
Indicator.eval(test_pred2['XGBoost2'], test_df_copy['Qobs'])

最佳参数组合：{'learning_rate': 0.15, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1}
*******************率定期*******************
MAE:125.72220531068149
RMSE:240.17724357539535
NSE:0.9682572519972895
RE:0.15871996982967906
Qmaxe:0.02390670687134503
Qmine:0.5561751108320933
*******************验证期*******************
MAE:178.62639074484252
RMSE:348.4104212489616
NSE:0.9442131807399858
RE:0.12397681675968615
Qmaxe:0.208197144500969
Qmine:0.9036705239305218


(178.62639074484252,
 348.4104212489616,
 0.9442131807399858,
 0.12397681675968615,
 0.208197144500969,
 0.9036705239305218)

In [9]:
"""
案例三：BP+RNN+XAJ+TANK
"""
# 处理数据
# 处理数据
trainy3 = train_df.iloc[:, :1]
trainx3 = pd.merge(trainx1, trainx2, on="Date", how="left")
testy3 = test_df.iloc[:, :1]
testx3 = pd.merge(testx1, testx2, on="Date", how="left")
train_data3=pd.merge(trainy3,trainx3,on="Date",how="left")
test_data3=pd.merge(testy3,testx3,on="Date",how="left")

# 数据归一化
scaler = MinMaxScaler(feature_range=(0, 1))
df_for_training_scaled3 = scaler.fit_transform(train_data3)
df_for_testing_scaled3 = scaler.transform(test_data3)

# 划分数据
trainX3 = df_for_training_scaled3[:, 1:]
trainY3 = df_for_training_scaled3[:, :1]
testX3 = df_for_testing_scaled3[:, 1:]
testY3 = df_for_testing_scaled3[:, :1]


parameters = {
    'max_depth': [3, 6, 10],
    'learning_rate': [0.01, 0.15, 0.025],
    'n_estimators': [50, 100, 200],
    'subsample':[ 0.7, 0.8, 0.9, 1]
}
xlf = xgb.XGBRegressor()
grid_search = GridSearchCV(xlf, param_grid=parameters, cv=5)
grid_search = grid_search.fit(trainX3, trainY3)
# 拟合数据集
model1 = grid_search.best_estimator_
print("最佳参数组合："+str(grid_search.best_params_))

train_predict3 = model1.predict(trainX3)
train_predict3 = pd.DataFrame(train_predict3)
train_predict3.columns = ["XGBoost3"]

test_predict3 = model1.predict(testX3)
test_predict3 = pd.DataFrame(test_predict3)
test_predict3.columns = ["XGBoost3"]


# 逆缩放过程
# 改变形状来进行逆缩放，逆变换后的第一列是我们需要的，所以我们在最后使用了 → [:,0]。
y_pred_copy = np.repeat(test_predict3,len(test_data3.columns), axis=-1)
test_pred3= scaler.inverse_transform(np.reshape(y_pred_copy, (len(test_predict3),len(test_data3.columns))))[:, 0]
test_pred_copy3 = test_pred3
test_pred3 = pd.DataFrame(test_pred3)
test_pred3.columns = ["XGBoost3"]

prediction_copies_array = np.repeat(train_predict3,len(test_data3.columns), axis=-1)
train_pred3 = scaler.inverse_transform(np.reshape(prediction_copies_array, (len(train_predict3),len(test_data3.columns))))[:, 0]
train_pred_copy3 = train_pred3
train_pred3=pd.DataFrame(train_pred3)
train_pred3.columns=['XGBoost3']


# 计算nash效率系数
print("*******************率定期*******************")
Indicator.eval(train_pred3['XGBoost3'], train_df_copy['Qobs'])
print("*******************验证期*******************")
Indicator.eval(test_pred3['XGBoost3'], test_df_copy['Qobs'])

最佳参数组合：{'learning_rate': 0.15, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1}
*******************率定期*******************
MAE:122.06047954044064
RMSE:224.81337122031127
NSE:0.9721884569517975
RE:0.16339882343047413
Qmaxe:0.03582156889619883
Qmine:0.6594978211418031
*******************验证期*******************
MAE:180.03341082635902
RMSE:347.3245215842442
NSE:0.9445603832388575
RE:1.0165826388028458
Qmaxe:0.20870208030523255
Qmine:1.0300652957657008


(180.03341082635902,
 347.3245215842442,
 0.9445603832388575,
 1.0165826388028458,
 0.20870208030523255,
 1.0300652957657008)

In [11]:
test_df_full = pd.concat([test_df_copy, test_pred1, test_pred2, test_pred3], axis=1)
test_df_full = test_df_full.reset_index(drop=True)
test_df_full = test_df_full.set_index('Date')
test_df_full.to_csv("/Users/datou/PycharmProjects/model/HengyangSta/HY_ensemble/data/HY_XGBoost1Test.csv")
train_df_full = pd.concat([train_df_copy, train_pred1, train_pred2, train_pred3], axis=1)
train_df_full = train_df_full.reset_index(drop=True)
train_df_full = train_df_full.set_index('Date')
train_df_full.to_csv("/Users/datou/PycharmProjects/model/HengyangSta/HY_ensemble/data/HY_XGBoost1Train.csv")

,Qobs,XGBoost1,XGBoost2,XGBoost3
Date,,,,
1985-01-16,479,788.259949,482.629578,511.886963
1985-01-17,542,788.259949,440.507172,474.851776
1985-01-18,557,788.259949,587.034485,619.759521
1985-01-19,476,771.178040,530.957886,563.682922
1985-01-20,464,714.305542,440.507172,472.252289
...,...,...,...,...
1996-11-20,374,339.411713,369.569916,371.115326
1996-11-21,315,339.411713,369.569916,371.115326
1996-11-22,300,339.411713,311.918915,316.701782
